In [1]:
import numpy as np
import pandas as pd 
from pandas.errors import EmptyDataError
import requests 
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re
from Bio.Alphabet import IUPAC
import subprocess
from collections import OrderedDict
import warnings
import os, os.path
import sys
import glob
import shutil
from Bio.SubsMat.MatrixInfo import blosum62
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
import sklearn.cluster as cluster
import sklearn 

pd.options.mode.chained_assignment = None
#Chrome Driver imports
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

/Users/evan/miniconda3/envs/malab/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
import os 
import pandas as pd
import glob

def create_directory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

def remove_thing(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)

def empty_directory(path):
    for i in glob.glob(os.path.join(path, '*')):
        remove_thing(i)

def create_run_directory(run_name,MSA=False):
    create_directory(run_name)
    create_directory(run_name+"/input")
    if MSA:
        create_directory(run_name+"/MSA_input")
        create_directory(run_name+"/MSA_output")
    create_directory(run_name+"/output")
    create_directory(run_name+"/summary")
    create_directory(run_name+"/run_params")

def write_run_params_file(config, spec_path, spec_hc):
    config_keys = ["RunName","GenesFile","odb_level"]
    run_name = config["RunName"]
    fpath = run_name+"/run_params/params.txt"
    params_f = open(fpath, 'wt')
    for key in config_keys:
        val = config[key]
        file_line = "{0}: {1}\n".format(key,val)
        params_f.write(file_line)
    params_f.write("species_list: {0}\n".format(spec_path))
    params_f.write("species_hashcode: {0}\n".format(spec_hc))
    params_f.close()

create_directory("tmp")
empty_directory('tmp')
print("Cleared tmp directory")

Cleared tmp directory


In [80]:


class Error(Exception):
    """Base class for exceptions in this module."""
    pass
class SequenceDataError(Error):
    error_type = "SequenceDataError"
    def __init__(self,code, message):
        self.code = code
        self.message = message
class OrthoDBQueryError(Error):
    error_type = "OrthoDBQueryError"
    def __init__(self, code, message):
        self.code = code
        self.message = message
class GeneCardsError(Error):
    error_type = "GeneCardsError"
    def __init__(self, code, message):
        self.code = code
        self.message = message
class SequenceAnalysisError(Error):
    error_type = "SequenceAnalysisError"
    def __init__(self, code, message):
        self.code = code
        self.message = message

        
def write_errors(errors_fpath,gene_name,error):
    import os.path
    error_type = error.error_type
    error_code = error.code 
    error_msg = error.message
    if not os.path.exists(errors_fpath):
        errors_f = open(errors_fpath,'wt')
        errors_f.write("gene\terror_type\terror_code\terror_str\n")
    else:
        errors_df = pd.read_csv(errors_fpath,delimiter='\t')
        if gene_name in errors_df["gene"].unique():
            gene_error_df = errors_df.loc[errors_df["gene"]==gene_name,:]
            if gene_error_df["error_str"].str.contains(error_msg).any():
#                 print("Previously stored error:")
                error_row = gene_error_df.loc[gene_error_df["error_str"]==error_msg,:]
                gene_name,error_type,error_code,error_msg = error_row.values[0]
                print("{0}\t{1}\t{2}\t{3}".format(gene_name,error_type,error_code,error_msg))
                return
    errors_f = open(errors_fpath,'at')
    fline = "{0}\t{1}\t{2}\t{3}\n".format(gene_name,error_type,error_code,error_msg)
    errors_f.write(fline)
    print(fline)
    errors_f.close()

In [4]:
# odb_error = OrthoDBQueryError(0,"No OrthoDB results for query")
# print(odb_error.error_type)

In [5]:
def parse_config(config_file="config/config.txt"):
    #Parse config text file (INI format) to establish paramters for the run
    #config_file should be a path to the config file ("config/config.txt" by default)
    import configparser
    config = configparser.ConfigParser()
    config.read(config_file)
    return config["DEFAULT"]
def parse_genes(genes_path="config/genes.txt"):
    #Parses gene file into list of uppercase, whitespace trimmed gene names
    gene_flines = open(genes_path).readlines()
    genes = [gene.strip().upper() for gene in gene_flines]
    return genes
def parse_species(species_path="config/v10_0_species.txt"):
    #Reads species list from file in config directory. Also returns a hashcode for the list of species used
    spec_lines = open(species_path).readlines()
    species = [spec.strip() for spec in spec_lines]
    concat = ""
    for spec in species: 
        concat = concat + spec
    hc = np.abs(hash(concat))
    return species, hc

def odb_tablev9(species_list,table_path="odb9v1_raw/odb9v1_species.tab"):
    odb = pd.read_csv(table_path,delimiter="\t",header=None,names=["tax_id","odb_id","spec_name","clustered_genes","ortho_groups","mapping_type"])
    filtered = pd.DataFrame(columns=odb.columns)
    for spec in species_list:
        row = odb[odb["spec_name"]==spec]
        filtered = filtered.append(row)
    filtered.drop(columns=["clustered_genes","ortho_groups","mapping_type"],inplace=True)
    return filtered
def odb_tablev10(species_list,table_path="odb10v0/odb10v0_species.tab"):
    """odb10v0_species.tab
    1.	NCBI tax id
    2.	Ortho DB individual organism id, based on NCBI tax id
    3.	scientific name inherited from the most relevant NCBI tax id
    4.	genome asssembly id, when available
    5.	total count of clustered genes in this species
    6.	total count of the OGs it participates
    7.	mapping type, clustered(C) or mapped(M)
    """
    odb = pd.read_csv(table_path,delimiter="\t",header=None,names=["tax_id","odb_id","spec_name","assembly_id","clustered_genes","ortho_groups","mapping_type"])
#     display(odb)
    filtered = pd.DataFrame(columns=odb.columns)
    for spec in species_list:
        row = odb[odb["spec_name"]==spec]
        filtered = filtered.append(row)
    filtered.drop(columns=["clustered_genes","ortho_groups","mapping_type"],inplace=True)
    return filtered

config = parse_config()
test_species = config["TestSpecies"]
species_path="config/v10_0_species.txt"
spec_list, hc = parse_species(species_path)
# gene_list = parse_genes("config/complexIV_V_genes.txt")
gene_list = parse_genes(config["GenesFile"])
tax_table = odb_tablev10(spec_list)
run_name = config["RunName"]
create_directory(run_name)
errors_fpath = run_name+'/summary/errors.tsv'
seq_qc_fpath = run_name+'/summary/seq_QC.tsv' 
DISPLAY_PARAMS = False
if DISPLAY_PARAMS:
    print("Tax table for species list at "+species_path)
    display(tax_table)
    print("Gene list: "+str(gene_list))
    print("Run Name: "+ run_name)
#Verify that species table, gene list, and run_name are correct

In [6]:
def gen_blos_df():
    global aas, blosum62_bg
    aas = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V','-']
    bg_probs = [0.078, 0.051, 0.041, 0.052, 0.024, 0.034, 0.059, 0.083, 0.025, 0.062, 0.092, 0.056, 0.024, 0.044, 0.043, 0.059, 0.055, 0.014, 0.034, 0.072]#, 0.000001]
    blosum62_bgdict = dict(zip(aas,bg_probs))
    blosum62_bg = bg_probs
    blos_df = pd.DataFrame(index=aas[:-1],columns=aas[:-1])
    for pair in blosum62:
        val = blosum62[pair]
        first, second = pair[0],pair[1]
        if first in aas and second in aas:
            blos_df.loc[first,second] = val
            blos_df.loc[second,first] = val
    sim_matrix = blos_df.values
    return aas, blosum62_bg, blos_df, sim_matrix


In [7]:
#Acquire input data via OrthoDB API 
def ODB_query(run_name,gene_name,level_str,spec_str):
    """Queries OrthoDB via the fasta and tab API for gene_name. 
    More info: https://www.orthodb.org/orthodb_userguide.html#api
    level_str corresponds to the API variable for phylogenetic clade 
    spec_str corresponds to the taxonomy ids for the list of species from the config folder 
    """
    import time, json 
    from json import JSONDecodeError
    #File paths and OrthoDB urls for downloads. NOTE BASE_URL might need updating depending on ODB conventions
    BASE_URL = "https://v101.orthodb.org"
    query_str = "query={0}".format(gene_name)
    fasta_url = "{0}/fasta?{1}&{2}&{3}".format(BASE_URL,query_str,level_str,spec_str)
#     print(fasta_url)
    fasta_path = "{0}/input/{1}.fasta".format(run_name,gene_name)
    tsv_url = "{0}/tab?{1}&{2}&{3}".format(BASE_URL,query_str,level_str,spec_str)
    tsv_path = "{0}/input/{1}.tsv".format(run_name,gene_name)
    #Obey OrthoDB download restrictions (one request per second) bc you're a good noodle
    t1 = time.process_time()
    fasta_proc = subprocess.run(args=['wget',fasta_url,'-O',fasta_path])
    if (time.process_time()-t1) < 1:
        time.sleep(0.5) #Gotta be nice to the orthodb servers or they block you :^( 
    t1 = time.process_time()
    tsv_proc = subprocess.run(args=['wget',tsv_url,'-O',tsv_path])
    if (time.process_time()-t1) < 1:
        time.sleep(0.5)
    try: 
        #JSON format returned if no results for query string - try opening downloaded data as JSON, if
        #successful, raise an OrthoDBQueryError
        tsv_json = json.load(open(tsv_path))
        os.remove(fasta_path)
        os.remove(tsv_path)
        raise OrthoDBQueryError(0,"No OrthoDB results for query")
    except JSONDecodeError:
        #Check if html syntax present in file; if not, query was successful and run_name/input should now
            #have ODB formatted .fasta and .tsv files
        file_txt = ""
        with open(fasta_path,"rt") as fasta_f:
            for i in range(10):
                file_txt = file_txt + fasta_f.readline()
            if bool(BeautifulSoup(file_txt,"html.parser").find()):
                os.remove(fasta_path)
                os.remove(tsv_path)
                raise OrthoDBQueryError(1,"OrthoDB search yielded too many clusters")
            #If no OrthoDBQueryError is raised, download was successful (no further action needed)
    
def download_input_data(gene_list,tax_table,config):
    """Queries OrthoDB for all entries in gene list (logs failed searches into errors_fpath), using species 
    list from tax_table and taxonomy level provided in config directory."""
    tax_ids = tax_table["tax_id"].values.astype(str)
    spec_str = "species="+",".join(tax_ids)
    level_str = "level="+str(config["odb_level"])
    failed_queries = []
    run_name = config["RunName"]
    errors_fpath = run_name+"/summary/errors.tsv"
    if os.path.exists(errors_fpath):
        errors_df = pd.read_csv(errors_fpath,delimiter='\t')
        ODB_errors_df = errors_df.loc[errors_df["error_type"]=="OrthoDBQueryError",:]
        check_error_file = True
    else:
        check_error_file = False
    for gene_name in gene_list: 
        fasta_path = "{0}/input/{1}.fasta".format(run_name,gene_name)
        if config.getboolean("OverwriteInput") or not os.path.exists(fasta_path):
            if check_error_file and gene_name in ODB_errors_df["gene"].unique():#ODB_errors_df["gene"].str.match(gene_name).any():
                ODB_error_row = ODB_errors_df.loc[ODB_errors_df["gene"]==gene_name,:]
                genename,error_type,error_code,error_msg = ODB_error_row.values[0]
                print("{0}\t{1}\t{2}\t{3}".format(genename,error_type,error_code,error_msg))
                failed_queries.append(gene_name)
            else:
                try:
                    ODB_query(run_name,gene_name,level_str,spec_str)
                except OrthoDBQueryError as odb_error:
                    failed_queries.append(gene_name)
                    write_errors(errors_fpath,gene_name,odb_error)

    print("Input queries downloaded.")
    valid_queries = [gene for gene in gene_list if gene not in failed_queries]
    return valid_queries, failed_queries

#Error Testing for bad OrthoDB queries
gene_list = parse_genes(config["GenesFile"])
# valid_searches, failed_queries = download_input_data(gene_list,tax_table,config) 
# testset1 = ["CDC42","OLFR710A","ATP5G1","CBR3-AS1"]
# download_input_data(testset1,tax_table,config)

In [8]:

def format_odb_field(field):
    """Remove spaces, commas, and capitalization from alias/ odb fields to search for string matches.
    If field is empty (np.nan), return empty string"""
    if(type(field)) == str:
        field = field.replace(" ","")
        field = field.replace(",","")
        field = field.replace("\n","")
        return field.lower()
    elif type(field) == float and np.isnan(field):
        return ""
    
def write_aliases_f(aliases,aliases_fpath):
    aliases_f = open(aliases_fpath,'wt')
    for a in aliases:
        aliases_f.write(a.strip()+'\n')
    aliases_f.close()
def download_alias_data(genename):
    """Queries GeneCards for alias data for genename. Should only be called if aliases_fpath doesn't exist 
    (ie if query has not been previously run and written to file). Attempts GeneCards query - if genename
    leads to a single entry page, pulls aliases from page html. If query leads to a query results page, 
    checks all linked entries to see if any contain genename. If none do (or other WebDriver issues arise),
    raises a GeneCardsError. 
    
    If query was successful (either single result page or successfully chose linked result from query results),
    return aliases and gc_name (the gene identifier used by GeneCards). gc_name stored separately since alias html 
    extraction will miss it otherwise. Also writes alias data to aliases_fpath
    
    Updated 01/10/2020. If function is consistently failing, check xpath class names against orthodb website"""
    aliases_fpath = "aliases_data/"+genename+"_aliases.txt"
    from selenium.webdriver.chrome.options import Options
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  
    WINDOW_SIZE = "1920,1080"
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    import time 
#         time.sleep(1.05)
#     gene_cards_url = "https://www.genecards.org/cgi-bin/carddisp.pl?gene="+genename.upper()+\
#                       "&keywords="+genename.upper()+"#aliases_descriptions"
    gene_cards_url = "https://www.genecards.org/cgi-bin/carddisp.pl?gene={0}".format(genename.upper())
    alias_left_xpath = "//div[@class='col-xs-12 col-sm-6 gc-double-column-desktop'][1]/ul/li"
    alias_right_xpath = "//div[@class='col-xs-12 col-sm-6 gc-double-column-desktop'][2]/ul/li"
    alias_single_xpath = "//div[@class='col-xs-8'][1]/ul/li"
#     elem_xpaths = [alias_left_xpath,alias_right_xpath,alias_single_xpath]
    list_xpath = "//ul[@class='list-unstyled list-spacious']/li"
    elem_xpaths = [list_xpath]
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(gene_cards_url)
    aliases = []
    for xpath in elem_xpaths:
        elems = driver.find_elements_by_xpath(xpath)
        innerHTMLs = [elem.get_attribute("innerHTML") for elem in elems]
        col_aliases = [BeautifulSoup(markup).find(text=True).strip() for markup in innerHTMLs]
        aliases.extend(col_aliases)
    if len(aliases) > 0:  
        #Means genename query to GeneCards autoredirected to a single page - normal aliases scraping
        #HTML parsing for GeneCards website - end result is list of trimmed alias strings
        gc_re = re.search("gene=([A-Z0-9]+)",gene_cards_url)
        gc_name = gc_re.groups()[0].strip()
        if gc_name not in aliases:
            aliases.insert(0,gc_name)
        #Cache aliases to aliases_fpath
        write_aliases_f(aliases,aliases_fpath)
        driver.quit()
        return aliases, gc_name
    else: 
        #Try search results page for genename; raise GeneCardsError if no results or check each page 
        #for alias matching genename otherwise
        query_url = "https://www.genecards.org/Search/Keyword?queryString={0}".format(gene_name)
        links_xpath = "//td[@class='gc-gene-symbol gc-highlight symbol-col']/a"
        link_elems = driver.find_elements_by_xpath(links_xpath)
        if link_elems: 
            for elem in links_elems:
                elem_href = elem.get_attribute("href")
                driver.get(elem_href)
                query_url = driver.current_url
                elem_gc_name = re.search("gene=([A-Z0-9]+)",query_url).groups()[0].strip()
                elem_aliases = []
                for xpath in elem_xpaths:
                    elems = driver.find_elements_by_xpath(xpath)
                    innerHTMLs = [elem.get_attribute("innerHTML") for elem in elems]
                    col_aliases = [BeautifulSoup(markup).find(text=True).strip() for markup in innerHTMLs]
                    elem_aliases.extend(col_aliases)
                if genename in elem_aliases or genename == elem_gc_name:
                    #Found query result with genename 
                    driver.quit()
                    if elem_gc_name not in elem_aliases:
                        elem_aliases.insert(0,elem_gc_name)
                    write_aliases_f(elem_aliases,aliases_fpath)
                    return elem_aliases, elem_gc_name
        #If either no link_elems (empty search results page), or none correspond to genename:
        driver.quit()
        raise GeneCardsError(0,"Could not automatically fetch alias data from GeneCards - consider searching manually")

def find_ref_seqs(genename, tsv_df,errors_fpath):
    """Returns a list of the orthodb ids of the reference sequences to use for distance/ length pre-filtering
    First attempts to find entries for which pub_gene_id is genename (works in most cases)
    In the event that none of the orthodb sequences have a pub_gene_id match, uses genecards to find aliases and run
    a more comprehensive search of the input for reference sequences to use. GeneCard aliases are saved in text
    files in input/aliases. 
    EDIT: Now uses GeneCard aliases regardless of number of pubgene_id annotated sequences""" 
    ref_ids = []
    #Go to genecards page for genename, extract information for aliases from the webpage/ html 
    aliases_fpath = "aliases_data/"+genename+"_aliases.txt"
    
    #If file doesn't exist or was improperly downloaded to yield only one line, repeat 
    #fetching alias names 
    if (not os.path.exists(aliases_fpath)) or len(open(aliases_fpath,'r').readlines()) == 1:
        try: 
            aliases, gc_name = download_alias_data(genename)
            matches = set((genename.upper(),gc_name.upper()))
        except GeneCardsError as gc_error:
            aliases = [genename]
            matches = set((genename.upper(),))
            write_errors(errors_fpath,genename,gc_error)
    else:
        #Read aliases information previously downloaded from GeneCards
        aliases_f = open(aliases_fpath,'r')
        aliases = aliases_f.readlines()
        aliases_f = open(aliases_fpath,'r')
        gc_name = aliases_f.readline().strip()
        matches = set((genename.upper(),gc_name.upper()))
    #Remove spaces, commas, new line chars, and capitalization from alias strings
    formatted_aliases = [format_odb_field(alias) for alias in aliases]
    #Search fields in search_fields for matches to the alias strings provided by GeneCards
    #Iterate tsv_df rows, save all reference ids which have matches 
    search_fields = ["pub_gene_id","og_name","description"]
    aliases_pat = "|".join(formatted_aliases)
    for idx,row in tsv_df.iterrows():
#         for field in search_fields:
            #Current behavior: exact matches in formatted pub_gene_id, og_name, or description only.
            #TODO: Add in partial string matching. Difficulties with distinguishing genenames
        for alias in formatted_aliases: 
            for field in search_fields:
                formatted_field = format_odb_field(str(row[field]))
                try:
                    if re.search(alias,formatted_field):
                        if idx not in ref_ids:
                            ref_ids.append(idx)#["int_prot_id"])
                            break
                except Exception as e:
                    #Special regexp characters present in alias 
                    if alias in formatted_field:
                        if idx not in ref_ids:
                            ref_ids.append(idx)#["int_prot_id"])
                            break
    #matches is a tuple of strings used to filter reference sequences in pg_id_df; either 
    #one entry or genename and then the entry used by the GeneCards page 
    return ref_ids, matches

# test_set = ["CLEC2D"]
# tmp_errors_fpath = "tmp/errors.tsv"
# for gene in test_set:

#     test_tsv_fpath = run_name+"/input/{0}.tsv".format(gene)
#     test_tsv = pd.read_csv(test_tsv_fpath,delimiter='\t')
#     test_tsv = test_tsv.set_index(keys="int_prot_id",drop=True)
#     display(test_tsv)

#     ref_ids, matches = find_ref_seqs(gene,test_tsv,tmp_errors_fpath)
    
#     ref_tsv = test_tsv.loc[ref_ids,:]
#     display(ref_tsv)
#     print(ref_ids)

#     print(matches)

# test_set2 = ["ATP5G"]#,"SADFGH"]
# for name in test_set2:
#     pass
#     print(download_alias_data(name))
#     download_alias_data(name)


In [9]:
#Fasta file reading functions: 
#filter_fasta_infile reads input files and outputs all records corresponding to filtered_ids to a new file
#read_fasta_lengths generates a Series object of lengths corresponding to the records in fasta_path 

def filter_fasta_infile(filtered_ids, infile_path, outfile_path=None,ordered=False):
    #If outfile_path is provided, write filtered fasta to outfile_path
    """Generates new fasta file to outfile_path using the subset of sequences in infile_path
    which have ids in filtered_ids
    ordered: if true, sequences will be returned/ written in order of filtered_ids
             if false, uses sequence order of sequences in infile_path
    """
    def filtered_generator(filtered_ids, infile_path):
        fasta_seqs = SeqIO.parse(open(infile_path),'fasta')
        for fasta in fasta_seqs:
            if fasta.id in filtered_ids:
                yield fasta 
    def ordered_filtered_generator(filtered_ids, infile_path):
        for id_ in filtered_ids:
            fasta_seqs = SeqIO.parse(open(infile_path),'fasta')
            for fasta in fasta_seqs:
                if fasta.id == id_:
                    yield fasta
                    break
    if outfile_path:
        if ordered:
            filtered = ordered_filtered_generator(filtered_ids, infile_path)
        else:
            filtered = filtered_generator(filtered_ids, infile_path)
        SeqIO.write(filtered,outfile_path,"fasta")
    if ordered:
        filtered = ordered_filtered_generator(filtered_ids, infile_path)
    else:
        filtered = filtered_generator(filtered_ids, infile_path)
    filtered_srs = pd.Series(index=filtered_ids)
    for fasta in filtered:
        filtered_srs[fasta.id] = str(fasta.seq)
    return filtered_srs


def srs_to_fasta(seq_srs, outfile_path):
    #Write records in seq_srs to outfile_path in fasta format 
    def record_generator(seq_srs):
        for idx, seq in seq_srs.iteritems():
            record = SeqRecord(Seq(seq,IUPAC.protein),id=idx)
            yield record
    records = record_generator(seq_srs)
    SeqIO.write(records,outfile_path,"fasta")

def read_fasta_lengths(fasta_path,aligned=False):
    #Reads fasta records from fasta_path, returns a Series or non-gap length
    #Also returns median and mode lengths for those records
    fasta_seqs = SeqIO.parse(open(fasta_path),'fasta')
    ids, lengths = [], []
    for fasta in fasta_seqs:
        ids.append(fasta.id)
        if aligned == True:
            seq_str = str(fasta.seq)
            non_gap_len = len(seq_str) - seq_str.count('-')
            lengths.append(non_gap_len)
        else:
            lengths.append(len(str(fasta.seq)))
    id_length_dict = dict(zip(ids,lengths))
    med_len = np.median(lengths)
    counts = np.bincount(lengths)
    mode_len = np.argmax(counts)
    return pd.Series(name="length",data=id_length_dict), med_len, mode_len

def fasta_to_srs(fasta_path):
    fasta_seqs = SeqIO.parse(open(fasta_path),'fasta')
    id_seq_map = OrderedDict()
    for fasta in fasta_seqs:
        record_id = fasta.id
        seq = str(fasta.seq)
        id_seq_map[record_id] = seq
    return pd.Series(name="seq",data=id_seq_map)

def align_srs_to_df(align_srs):
    #Returns DataFrame object from series of aligned sequences; columns are 1-indexed positions
    #Values are characters in alignment, index is ODB sequence IDs
    n_seq = len(align_srs)
#     display(align_srs)
#     display(align_srs.iloc[0])
    seq_len = len(align_srs.iloc[0])
    align_df = pd.DataFrame(index=align_srs.index,columns=range(seq_len))
    for idx, seq in align_srs.iteritems():
        align_df.loc[idx,:] = list(seq)
    align_df.columns += 1
    return align_df

def seq_srs_to_align_df(seq_srs,align_in_fpath,align_out_fpath):
    """Transform seq_srs (pandas Series containing sequence texts) to a DataFrame for which each column
    is an alignment position and column. Writes input fasta and output fastas for alignment to align_in_fpath
    and align_out_fpath respectively. Also returns average (non-diagonal) identity distances"""
    srs_to_fasta(seq_srs,align_in_fpath)
    n, ordered_ids, id_dm, align_srs = construct_id_dm(seq_srs,align_in_fpath,align_out_fpath)
    align_df = align_srs_to_df(align_srs)
    dist_srs = avg_dist_srs(align_srs.index,id_dm)
    return align_df, dist_srs
    
def align_srs_to_seq_srs(align_srs,outfile_path=None):
    #Return new Series (same index) of sequences with gap characters dropped
    #If outfile_path is provided, write un-aligned record seqs to new fasta file
    seq_srs = pd.Series(index=align_srs.index)
    for idx, align_seq in align_srs.iteritems():
        seq = align_seq.replace("-","")
        seq_srs[idx] = seq
    if outfile_path:
        srs_to_fasta(seq_srs,outfile_path)
    return seq

def align_df_to_srs(align_df):
    #Returns series of aligned sequences from array of aligned positions
    align_srs = pd.Series(index=align_df.index)
    for idx,record in align_df.iterrows():
#       #seq is a string joining all characters with no delimiter (i.e. the original aligned sequence with gaps)
        seq = ''.join(record.values)
        align_srs[idx] = seq
    return align_srs

def truncate_align_df(align_df, record_id):
    #Truncates align_df to only the positions corresponding to positions in the sequence of record_id
    #Returns truncated_df: new DataFrame of these positions (with reset positions)
    # len_adjust_srs: Series with same index as align_df containing counts of non-gap characters dropped 
    # to get sequences present in truncated_df 
    record_srs = align_df.loc[record_id]
    for index,entry in record_srs.iteritems():
        if entry != '-':
            trunc_idx, trunc_entry = index, entry   
            break
    truncated_df = align_df.loc[:,trunc_idx:]
    if trunc_idx > 1:
        #If some align positions being truncated, iterate over align_df and determine length of sequences
        # (i.e. non-gap character count) per record in the truncated fraction of the alignment 
        pre = align_df.loc[:,:trunc_idx-1]
        len_adjust_srs = pd.Series(index=align_df.index)
        for index,record in pre.iterrows():
            count = 0 
            for entry in record:
                if entry != '-':
                    count += 1
            len_adjust_srs[index] = count
        #Adjust positions in truncated_df
        first_pos = truncated_df.columns[0]
        offset = first_pos - 1
        truncated_df.columns -= offset
    else:
        #No truncation necessary to align to shortest known record (length adjustments = 0 for all)
        len_adjust_srs = pd.Series(index=align_df.index,data=np.zeros(len(align_df)))
    return truncated_df, len_adjust_srs

In [10]:
def load_ka_distmat(fasta_infile,align_outfile="tmp/ka_distmat_align.fasta",distmat_file="tmp/ka_distmat.tsv"):
    """REQUIRES: Modified KAlign source code to include distance matrix output, written to tmp/ka_distmat.tsv
    Uses KAlign's modified distance metric (Wu-Manber, partial scoring for 3aa patterns with one error 
    tolerated), outputs to tsv and reads and stores as an ndarray (n x n) with n = number of sequences in fasta_infile
    
    Also return a list of the ODB ids of sequences (in order) corresponding to the distmat/ alignment order
    """
    proc = subprocess.run(args=["kalign",'-i',fasta_infile,"-o",align_outfile,"-f","fasta"])
    distmat_flines = open(distmat_file).readlines()
    n = len(distmat_flines)
    distmat = np.ndarray((n,n))
    
    for i,line in enumerate(distmat_flines):
        as_list = line.split()
        line_arr = np.array(as_list).astype(np.float)
        distmat[i] = line_arr
    ordered_ids = []
    align_seqs = SeqIO.parse(open(align_outfile),'fasta')
    for record in align_seqs:
        ordered_ids.append(record.id)
    return n, ordered_ids, distmat, align_outfile
def construct_id_dm(seq_df,seq_fpath,align_outpath="tmp/iddm_align.fasta",ordered=False):
    from Bio.Phylo.TreeConstruction import DistanceCalculator
    from Bio import AlignIO
    #Filter records in seq_fpath to new fasta only containing records in seq_df.index
    filtered_outpath = "tmp/iddm.fasta"
    filter_fasta_infile(seq_df.index,seq_fpath,outfile_path=filtered_outpath,ordered=ordered)
    #KAlign sequences in filtered_outpath, write to align_outpath
    n, ordered_ids, ka_dm, align_outfile = load_ka_distmat(filtered_outpath,align_outfile=align_outpath)
    align_srs = fasta_to_srs(align_outpath)
    aln = AlignIO.read(open(align_outpath), 'fasta')
    calculator = DistanceCalculator('identity')
    id_dm_obj = calculator.get_distance(aln)
    #Convert AlignIO object to np.ndarray
    for i,r in enumerate(id_dm_obj):
        if i == 0:
            id_dm = np.array(r)
        else:
            id_dm = np.vstack((id_dm,r))
    return n, ordered_ids, id_dm, align_srs
def avg_dist_srs(index,distmat):
    #index is a pandas Index object with entries corresponding to the distmat (i.e. lengths should be equal)
    #Calculate mean of non-self record distances (pairwise distance with self is 0, so sum functions as intended)
    n = len(distmat)
    avg_dists = np.sum(distmat, axis=1)/(n-1)
    dist_srs = pd.Series(data=avg_dists,index=index,name="dist")
    return dist_srs

def load_dist_length_data(ref_ids,ref_fasta_path,ref_tsv,aligned=False):
    #Returns a dataframe based off of ODB tsv files but with sequence length and average KAlign distance included
    metrics = {}
    n, ordered_ids, distmat, align_outfile = load_ka_distmat(ref_fasta_path)
    metrics["n"] = n
    ref_align_series = fasta_to_srs(align_outfile)
    lengths, med_len, mode_len = read_fasta_lengths(ref_fasta_path,aligned)
    metrics["median_len"] = med_len
    metrics["mode_len"] = mode_len
    #Average non-diagonal (which is always 0) distmat entries
    ref_dist_srs = avg_dist_srs(ordered_ids, distmat)
    #Incorporate average dists into ref_df
    ref_df = pd.DataFrame(index=ordered_ids,data={"dist":ref_dist_srs,"length":lengths, \
                                                             "align":ref_align_series})
    ref_df = ref_df.merge(ref_tsv,how="inner",left_index=True,right_index=True)
    if "level_taxid" in ref_df:
        ref_df.drop(columns="level_taxid",inplace=True)
    reordered_cols = ["dist","length","pub_og_id","og_name","organism_taxid","organism_name","pub_gene_id","description","align"]
    ref_df = ref_df[reordered_cols]
    return ref_df, metrics


In [11]:
# upper_matches = ["CALM1","ATP5G"]
# upper_matches = [match+"$|"+match+"[^\w]" for match in upper_matches]
# target_strs = ["CALM1;CALM2;CALM3", "CALM1","CALM2;CALM1", "ATP5G", "ATP5G;SLAD","SADKFJG;ATP5G", "CALM12"]
# pat = "|".join(upper_matches)
# print(pat)
# for target in target_strs:
#     print(re.search(pat, target))

In [12]:
def pg_id_match_df(unfiltered_df,matches):
    """From an unfiltered_df of reference sequences, returns a DataFrame of all entries which have pub_gene_id
    matching genename. If any species has more than one such sequence, only one entry will be kept, selected
    by the lowest average distance to the entire reference set. 
    """
#     display(unfiltered_df)
    upper_matches = [match.upper() for match in matches]
    upper_matches = [match+"$|"+match+"[^\w]" for match in upper_matches]
#     pg_id_df = unfiltered_df.loc[unfiltered_df["pub_gene_id"].str.upper().isin(upper_matches)]
    pat = "|".join(upper_matches)
    pg_id_df = unfiltered_df.loc[unfiltered_df["pub_gene_id"].str.upper().str.contains(pat)]
    #Filtering for any species with more than one sequence
    final_df = pd.DataFrame(columns=pg_id_df.columns)
    for unique_tax in pg_id_df["organism_taxid"].unique():
        spec_df = pg_id_df.loc[pg_id_df["organism_taxid"] == unique_tax]
        if len(spec_df) > 1:
            spec_df_index = spec_df.index
            min_dist_idx = spec_df["dist"].values.argmin()
            min_dist_row = spec_df.iloc[min_dist_idx,:]
            final_df = final_df.append(min_dist_row)
            #Remove all entries for this tax_id and append only the minimum distance row
#             pg_id_df.drop(spec_df.index,inplace=True)
#             pg_id_df = pg_id_df.append(min_dist_row)
        else:
            final_df = final_df.append(spec_df)
    return final_df

def final_input_df(pg_id_df,unfiltered_df):
    #Determine sets of values for NCBI taxid/ ODB description for which records exist in pg_id_df
    #i.e. ODB input records with pub_gene_id had matches to genename or its aliases
    #Final DataFrame will have extra column "" noting the type of annotation used to determine record validity
    pg_id_tax_set = pg_id_df["organism_taxid"].unique()
    pg_id_desc_set = set([format_odb_field(desc) for desc in pg_id_df["description"].unique()])
    filtered_df = pd.DataFrame(columns=pg_id_df.columns.append(pd.Index(["record_type"])))
#     filtered_df = pd.DataFrame(columns=pd.Index(["record_type"]).append(pg_id_df.columns))
    threshold_dist = pg_id_df["dist"].max()
    for ref_tax in unfiltered_df["organism_taxid"].unique():
        if ref_tax in pg_id_tax_set:
            #Check lengths against rest of pg_id_df
            pg_row = pg_id_df.loc[pg_id_df["organism_taxid"] == ref_tax]
            pg_row["record_type"] = "pubgene_id match"
            filtered_df = filtered_df.append(pg_row)
            continue 
        else:
            #Check if any of entries have description matching Gene annotated sequences
            spec_df = unfiltered_df[unfiltered_df["organism_taxid"] == ref_tax]
            desc_match_srs = spec_df["description"].apply(format_odb_field).isin(pg_id_desc_set)
            desc_match_df = spec_df.loc[desc_match_srs,:]
            #If records for this taxid have matches with the description values (pg_id_desc_set), 
            #Choose the min distance entry from desc_match_df
            #Else choose the min distance entry for this tax_id (from spec_df)
            if desc_match_df.empty:
                min_dist_idx = spec_df["dist"].values.argmin()
                record_type = "No ODB match"
            else:
                min_dist_idx = desc_match_df["dist"].values.argmin()
                record_type = "ODB description match"
            min_dist_row = spec_df.iloc[min_dist_idx,:]
            min_dist_row["record_type"] = record_type
            if min_dist_row["dist"] < threshold_dist:
                filtered_df = filtered_df.append(min_dist_row)
    return filtered_df

In [13]:
def filter_ref_seqs4(gene_name,matches,ref_fasta_path,ref_df,seq_qc_fpath,known_spec_list=["10090_0","9606_0","43179_0"]):
    ref_ids = list(ref_df.index)
#     ksr_tsv_df = ref_tsv.loc[ref_tsv["organism_taxid"].isin(known_spec_list)]
    ksr_ref_df = ref_df.loc[ref_df["organism_taxid"].isin(known_spec_list)]
    #Create new tmp fasta file with known species annotated records only
    ks_refseqs_fpath = "tmp/ks_refseqs.fasta"
    known_spec_records = filter_fasta_infile(ksr_ref_df.index,ref_fasta_path,outfile_path=ks_refseqs_fpath)
    
    #Search ksr_full_df for pub_gene_id field matches to main aliases (matches)
    upper_matches = ["{0}$|{0}[;]".format(match.upper()) for match in matches]
    matches_pat = "|".join(upper_matches)
    ksr_pgid_df = ksr_ref_df.loc[ksr_ref_df["pub_gene_id"].str.upper().str.contains(matches_pat)]
    final_ksr_df = select_known_species_records(ksr_pgid_df,ksr_ref_df,ks_refseqs_fpath)
    #Quality checking of final_ksr_df:
    final_ksr_df_QC(gene_name,matches,seq_qc_fpath,final_ksr_df,known_spec_list)
    ref_pgid_df = ref_df.loc[ref_df["pub_gene_id"].str.upper().str.contains(matches_pat)]
    final_df = select_species_records(ref_pgid_df,ref_df,final_ksr_df,ref_fasta_path)
    length_filter = True
    if length_filter:
        #Remove records whose length differs by more than 10% from the median (but keep representative seqs for 
        #human, mouse, GS)
        med_len = final_df["length"].median()
        len_filtered = final_df.loc[(np.abs((final_df["length"]-med_len)/med_len) < 0.1) | (final_df["organism_taxid"].isin(known_spec_list)),:]
        final_df = len_filtered
    return final_df

def select_known_species_records(ksr_pgid_df,ksr_ref_df,ks_refseqs_fpath):
    """Return a dataframe of at most one record per species in KS_taxids, selecting first if there is only 
    one pubgene_id match record for that species and otherwise selecting the record with maximum identity
    with other single pgid matched records, preferable from pgid_df but from ksr_ref_df if no pgid match exists
    for that species"""
    TEST_ID = "43179_0"
    KS_TAXIDS = ["10090_0","43179_0","9606_0"]
    ksr_taxid_uniques = ksr_ref_df["organism_taxid"].unique()
    pgid_taxid_uniques = ksr_pgid_df["organism_taxid"].unique()
    
    #Case Handling: Only one species in ksr_pgid
    if len(ksr_taxid_uniques) == 1 and TEST_ID in ksr_taxid_uniques:
        raise SequenceDataError(3,"No Human or Mouse reference sequences")
            
    #Case Handling - if ksr_pgid_df is not composed of one sequence each for human, mouse, 13LGS
    if len(ksr_pgid_df) > 3 or len(pgid_taxid_uniques) <3:
        single_match_pgid_records = pd.DataFrame(columns=ksr_pgid_df.columns)
        for ks_id in KS_TAXIDS:
            ks_pgid_df = ksr_pgid_df.loc[ksr_pgid_df["organism_taxid"]==ks_id,:]
            if len(ks_pgid_df) == 1:
                single_match_pgid_records = single_match_pgid_records.append(ks_pgid_df)
                
        if single_match_pgid_records.empty:
            #Case handling if no single_match_records (ie CALM1): 
            #Allow manual input selection from either ksr_pgid_df or ksr_ref_df, raise SequenceDataError if both empty
            if not ksr_pgid_df.empty: 
                print("pubgeneID matched records")
                display(ksr_pgid_df)
                selection_df = ksr_pgid_df
            elif not ksr_ref_df.empty:
                print("GeneCards alias matched records")
                display(ksr_ref_df)
                selection_df = ksr_ref_df
            else:
                raise SequenceDataError(2,"No GeneCards alias matched sequence records for human/mouse/GS")
            try: 
                input_idx = input("Enter 0-indexed position of representative sequence for analysis")
                int_idx = int(input_idx)
                selection_row = selection_df.iloc[int_idx,:]
                
            except (IndexError, ValueError) as e:
                print("Bad Input")
                while (not re.search("^\d+$",input_idx)) or (int(input_idx)>=len(selection_df) or int(input_idx)<0):
                    input_idx = input("Enter a number between 0 and {0}".format(len(selection_df)-1))
                int_idx = int(input_idx)
                selection_row = selection_df.iloc[int_idx,:]
            single_match_pgid_records = single_match_pgid_records.append(selection_row)

        final_ksr_df = pd.DataFrame(columns=single_match_pgid_records.columns)
        sm_record_ids = single_match_pgid_records.index                   
            
        for ks_id in KS_TAXIDS:
            if ks_id not in single_match_pgid_records["organism_taxid"].unique():
            #For known_species taxids with 0 or 2+ pgid records: first check pgid matches (2+ pgid), then ref seqs (0 pgid)
                if ks_id in pgid_taxid_uniques:
                    #2+ pgid records: Construct id_dm of pgid matched records; select best ks_id 
                    #based on max identity with single_match_records
                    n, ordered_ids, id_dm, align_srs = construct_id_dm(ksr_pgid_df,ks_refseqs_fpath)
                elif ks_id in ks_id in ksr_taxid_uniques:
                    #0 pgid records; construct id_dm from ksr_ref_df records, select max identity record
                    n, ordered_ids, id_dm, align_srs = construct_id_dm(ksr_ref_df,ks_refseqs_fpath)
                else:
                    continue
                #Maximum identity = minimum id_dm value based on AlignIO implementation
                md_row, min_dist = min_dist_spec_record(ks_id,id_dm,ordered_ids,sm_record_ids,ksr_ref_df)
                final_ksr_df = final_ksr_df.append(md_row)
            else:
                sm_row = single_match_pgid_records.loc[single_match_pgid_records["organism_taxid"]==ks_id,:]
                final_ksr_df = final_ksr_df.append(sm_row)
        return final_ksr_df
                
    else:
        #Sort ksr_pgid_df to order of taxids in KS_TAXIDS
        final_ksr_df = pd.DataFrame(columns=ksr_pgid_df.columns)
        for tax_id in KS_TAXIDS:
            row = ksr_pgid_df.loc[ksr_pgid_df["organism_taxid"]==tax_id,:]
            final_ksr_df = final_ksr_df.append(row)
        return final_ksr_df
    
def select_species_records(ref_pgid_df,ref_df,final_ksr_df,refseqs_fpath):
    TEST_ID = "43179_0"
    KS_TAXIDS = ["10090_0","43179_0","9606_0"]
    pgid_taxids = [tax_id for tax_id in ref_pgid_df["organism_taxid"].unique() if tax_id not in KS_TAXIDS]
    ref_taxids = [tax_id for tax_id in ref_df["organism_taxid"].unique() if tax_id not in KS_TAXIDS]
    
    #Distance calculations for final set of known species records - check internal identity values
    #Set identity threshold - other species sequences above this value will not be included
    ksr_dm_fpath = "tmp/ksr_dm_ka.fasta"
    n, ksr_ordered_ids, ksr_id_dm, ksr_align_srs = construct_id_dm(final_ksr_df,refseqs_fpath,ksr_dm_fpath,ordered=True)
    non_diagonal_avg = ksr_id_dm.sum(axis=0)/(n-1)
    max_dist_idx = non_diagonal_avg.argmax()
    max_dist_id = ksr_ordered_ids[max_dist_idx]
    max_dist = non_diagonal_avg[max_dist_idx]
    identity_threshold = np.mean(non_diagonal_avg)*1.5
    
    final_df = final_ksr_df.copy()
    unfiltered_df = final_ksr_df.copy()
    for tax_id in ref_taxids:
        try:
            tax_df = ref_df.loc[ref_df["organism_taxid"]==tax_id]
            tax_records = tax_df.index
            #tax_dm_filtered_ids: list of record ids in final_ksr_df followed by all records corresponding to tax_id
            tax_dm_filtered_ids = list(final_ksr_df.index)
            tax_dm_filtered_ids.extend(tax_records) 
            tax_dm_df = ref_df.loc[tax_dm_filtered_ids,:]
#             tax_dm_df.drop_duplicates(inplace=True)
            tax_dm_df = tax_dm_df.loc[~tax_dm_df.index.duplicated(keep='first')]
            tax_dm_fpath = "tmp/{0}_dm.fasta".format(tax_id)
            n, tax_ordered_ids, tax_id_dm, tax_align_srs = construct_id_dm(tax_dm_df,refseqs_fpath,tax_dm_fpath,ordered=True)
            md_row, min_dist = min_dist_spec_record(tax_id,tax_id_dm,tax_ordered_ids,final_ksr_df.index,tax_dm_df)
            if min_dist <= identity_threshold:
                final_df = final_df.append(md_row)
        except ValueError as e:
            #Debugging edge case for OrthoDB data error with duplicate entries (Irf2bp2)
            print(e)
            display(tax_dm_df)
            raise SequenceDataError(5,"Duplicate Sequence Entry")
    return final_df    
        
def final_ksr_df_QC(gene_name,matches,seq_qc_fpath,final_ksr_df,ks_taxids):
    #Writes warnings about compiled final known species records (ie after select_known_species_records)
    #to specified file path
    if len(final_ksr_df) < len(ks_taxids):
        for tax_id in ks_taxids:
            if tax_id not in final_ksr_df["organism_taxid"].unique():
                message_txt = "No reference sequence for tax_id: {0}".format(tax_id)
                write_ref_seq_QC(seq_qc_fpath,gene_name,message_txt)
    length_srs = final_ksr_df["length"]
    median_len = length_srs.median()
    for record_id in final_ksr_df.index:
        id_len = length_srs[record_id]
        if (np.abs(id_len-median_len)/median_len) >= 0.1:
            message_txt = "Record_id {0} has length {1} which is greater than 10% different from the median ({2})".format(record_id,id_len,median_len)
            write_ref_seq_QC(seq_qc_fpath,gene_name,message_txt)
    upper_matches = [match.upper() for match in matches]
    upper_matches = [match+"$|"+match+"[;]" for match in upper_matches]
    pat = "|".join(upper_matches)
#     final_pgid_df = final_ksr_df.loc[final_ksr_df["pub_gene_id"].str.upper().str.contains(pat)]
    for record_id,pgid in final_ksr_df["pub_gene_id"].iteritems():
        if not re.search(pat,pgid.upper()):
            message_txt = "Record_id {0} has pub_gene_id {1} which doesn't match gene_name ({2})".format(record_id,pgid,gene_name)
            write_ref_seq_QC(seq_qc_fpath,gene_name,message_txt)
def write_ref_seq_QC(seq_qc_fpath,gene_name,message):
    """Writes warning messages to seq_qc_fpath and prints (will not write duplicate entries)"""
    if not os.path.exists(seq_qc_fpath):
        seq_qc_f = open(seq_qc_fpath,'wt')
        seq_qc_f.write("gene\tmessage\n")
    else:
        seq_qc_f = open(seq_qc_fpath,'at')
        seq_qc_df = pd.read_csv(seq_qc_fpath,delimiter='\t')
        if gene_name in seq_qc_df["gene"].unique():
            gene_df = seq_qc_df[seq_qc_df["gene"]==gene_name]
            if message not in gene_df["message"].unique():
                fline = "{0}\t{1}\n".format(gene_name,message)
                seq_qc_f.write(fline)
            else:
                stored_message = seq_qc_df[seq_qc_df["gene"]==gene_name]["message"].iloc[0]
                print("Cached QC Warning:")
                message = stored_message
        else:
            fline = "{0}\t{1}\n".format(gene_name,message)
            seq_qc_f.write(fline)
    print("{0}: {1}".format(gene_name,message))

    
def min_dist_spec_record(spec_taxid,distmat,dm_record_ids,accepted_record_ids, ref_df):
    """Given a species taxid (spec_taxid), an np.ndarray distance matrix (distmat), a list of record_ids
    corresponding to the rows in distmat (dm_record_ids),a list of accepted record ids (accepted_record_ids)
    against which record distances will be averaged, and a DataFrame of sequences (ref_df):
    Calculates the average distance of every record containing spec_taxid against accepted records, then
    returns the row from ref_df corresponding to the record with lowest average distance""" 
    spec_records = [(i,id_) for i,id_ in enumerate(dm_record_ids) if re.search(spec_taxid,id_)]
    spec_dm_idxs = [t[0] for t in spec_records] 
    accepted_records = [(i,id_) for i,id_ in enumerate(dm_record_ids) if id_ in accepted_record_ids]
    accepted_dm_idxs = [t[0] for t in accepted_records]

    spec_dm = distmat[:,spec_dm_idxs]
    sub_dm = spec_dm[accepted_dm_idxs,:]
    if len(sub_dm) > 1:
        avg_dist = sub_dm.mean(axis=0)
    else:
        avg_dist = sub_dm[0]
    min_idx = np.argmin(avg_dist)
    min_dist_id = spec_records[min_idx][1]
    min_dist = avg_dist[min_idx]
    md_row = ref_df.loc[min_dist_id,:]
    return md_row, min_dist


#===================== 
test_set_1 = ["CALM1","CALCOCO2"]
# gene_set_1 = gene_list
# gene_set_1 = ["VMP1","CALM1"]
# errors_fpath = "tmp/errors.tsv"
# for gene_name in test_set_1[:]:
#     fasta_path = run_name+"/input/"+gene_name+".fasta"
#     tsv_path = run_name+"/input/"+gene_name+".tsv"
#     ref_fasta_path = "tmp/ref_seqs.fasta"
    
#     try: 
#         tsv_df = pd.read_csv(tsv_path,delimiter='\t')
#         unfiltered_n = len(tsv_df["organism_taxid"].unique())
#         tsv_df = tsv_df.set_index(keys="int_prot_id",drop=True)#drop=False)
#         ref_ids, matches = find_ref_seqs(gene_name,tsv_df,errors_fpath)
#         if len(ref_ids) == 0:
#             write_errors(errors_fpath,gene_name,1)
#         elif test_species not in tsv_df["organism_name"].values:
#             write_errors(errors_fpath,gene_name,2)
#         ref_tsv = tsv_df.loc[ref_ids,:]
#         filtered_seq_srs = filter_fasta_infile(ref_ids,fasta_path,outfile_path=ref_fasta_path)
#         ref_seq_df = pd.DataFrame(filtered_seq_srs,columns=["seq"])
#         ref_seq_df["length"] = [len(seq) for seq in ref_seq_df["seq"]]
#         ref_df = ref_tsv.join(ref_seq_df,how="inner")
#         if test_species not in ref_tsv["organism_name"].values:
#             raise SequenceDataError(4,"Test Species has no reference sequence in input (but a record is present in unfiltered ODB query)")
#         final_records_df = filter_ref_seqs4(gene_name,matches,ref_fasta_path, ref_df,seq_qc_fpath)
# #         final_align_df = seq_srs_to_align_df(final_records_df["seq"],)
#     except Exception as e:
#         print("==={0}===".format(gene_name))
#         print(e)


In [14]:
# print(id_dm)

In [15]:
#Ref Sequence Testing 
gene_set_1 = ["CALM1","ATP5G1"]

def process_input(gene_name, run_name, test_species, errors_fpath, seq_qc_fpath,drop_spec_list=None):
    #Generates final record and final alignment dataframes from raw input files. 
    #If errors arise (see write_errors), updates errors_fpath accordingly

    input_fasta_fpath = "{0}/input/{1}.fasta".format(run_name,gene_name)
    input_tsv_fpath = "{0}/input/{1}.tsv".format(run_name,gene_name)
    gene_output_dir = "{0}/output/{1}".format(run_name,gene_name)
    create_directory(gene_output_dir)
    MSA_input_fasta = "{0}/output/{1}/{1}.fasta".format(run_name,gene_name)
    MSA_output_fasta = "{0}/output/{1}/{1}_MSA.fasta".format(run_name,gene_name)
    ref_fasta_path = "tmp/ref_seqs.fasta"
    try: 
        tsv_df = pd.read_csv(input_tsv_fpath,delimiter='\t')
#         unfiltered_n = len(tsv_df["organism_taxid"].unique())
        tsv_df = tsv_df.set_index(keys="int_prot_id",drop=True)#drop=False)
    except (EmptyDataError, FileNotFoundError) as e:
        raise OrthoDBQueryError(0,"No OrthoDB results for query")
    if drop_spec_list:
        tsv_df = tsv_df.loc[~tsv_df["organism_taxid"].isin(drop_spec_list),:] 
    ref_ids, matches = find_ref_seqs(gene_name,tsv_df,errors_fpath)
    if len(ref_ids) == 0:
        raise SequenceDataError(0,"No reference sequences could be found")
    elif test_species not in tsv_df["organism_name"].values:
        raise SequenceDataError(1,"Test Species has no sequence in input")
    ref_tsv = tsv_df.loc[ref_ids,:]
    
    filtered_seq_srs = filter_fasta_infile(ref_ids,input_fasta_fpath,outfile_path=ref_fasta_path)
    ref_seq_df = pd.DataFrame(filtered_seq_srs,columns=["seq"])
    ref_seq_df["length"] = [len(seq) for seq in ref_seq_df["seq"]]
    ref_df = ref_tsv.join(ref_seq_df,how="inner")
    
    if test_species not in ref_tsv["organism_name"].values:
        raise SequenceDataError(4,"Test Species has no reference sequence in input (but a record is present in unfiltered ODB query)")
    final_records_df = filter_ref_seqs4(gene_name,matches,ref_fasta_path, ref_df,seq_qc_fpath)
    final_align_df, dist_srs = seq_srs_to_align_df(final_records_df["seq"],MSA_input_fasta,MSA_output_fasta)
    final_records_df.loc[:,"dist"] = dist_srs

#     print("Final Records DataFrame")
#     display(final_records_df)
#     print("Final Alignment DataFrame")
#     display(final_align_df)
    return final_records_df, final_align_df


In [43]:


def find_uniques(align_df, sub_freq_threshold, test_species_idx,display_uniques):
    #align
    uniques = pd.DataFrame(index=align_df.index)
    for pos in align_df:
        col = align_df[pos]
        sub_counts = col.value_counts()
        c_counts = sub_counts.value_counts()
        for index,value in sub_counts.iteritems():
    #         if(value == 1):# and c_counts.loc[value] == 1:
            if value <= sub_freq_threshold:
                if col[test_species_idx] == index and index != 'X':
                    uniques.loc[:,pos] = col
                    break
    if display_uniques:
        print("Threshold Number of Sequences: "+str(int(sub_freq_threshold)))#/len(ordered)))
        print("Species specific residues for "+test_species)
        display(uniques)
    return uniques

# uniques = find_uniques(align_df,sub_freq_threshold, display_uniques)
# display(align_df)
# display(uniques)


In [21]:
# def force_positions(align_df, test_species, positions):
# def force_insertion(align_df,species,pos,sub):
def gap_ratio(col):
    return sum([c == '-' for c in col])/len(col)

def sub_ratio(col,sub):
    return sum([c == sub for c in col])/len(col)

def species_gaps_dict(align_df,gap_char='-'):
    gaps_dict = OrderedDict()
    unique_gaps = OrderedDict()
    unique_subs = OrderedDict()
    align_nd = align_df.values
    for j,spec in enumerate(align_df.index):
        gaps_dict[spec] = []
        unique_gaps[spec] = []
        unique_subs[spec] = []
        spec_row = align_df.loc[spec,:].values
#         row = align_nd[j]
        for i,entry in enumerate(spec_row):
#             outgroup_col_ = align_df.iloc[:,i].copy()
            outgroup_col_ = align_nd.T[i].copy()
#             outgroup_col_.drop(spec,inplace=True)
            outgroup_col_ = np.delete(outgroup_col_,obj=j)
            ratio = gap_ratio(outgroup_col_)
            if entry == gap_char:
                gaps_dict[spec].append(i+1)
                #Check if remainder of positions have gap; if none do, add to unique gaps.
                if ratio <= 0.1:
                    unique_gaps[spec].append(i+1)
            else:
                if ratio >= 0.9:
                    unique_subs[spec].append(i+1)
                     
    return gaps_dict, unique_gaps, unique_subs

def seq_to_align_pos(seq_pos,species,gap_dict):
    spec_gaps = gap_dict[species]
    for gap in spec_gaps:
        if gap <= seq_pos:
            seq_pos += 1 
    return seq_pos

def outgroup_seq_pos(gaps,test_pos,test_species,out_species):
    test_gaps = gaps[test_species]
    out_gaps = gaps[out_species]
    out_pos = test_pos
    for test_gap in test_gaps:
        if test_gap <= out_pos and test_gap not in out_gaps:
            out_pos += 1
    for out_gap in out_gaps:
        if out_gap <= out_pos and out_gap not in test_gaps:
            out_pos -= 1
#             print("hurray")
    return out_pos
            

def force_noshift(align_df,test_species,subs):
    test_row = align_df.loc[test_species,:]
    for pos in subs:
        sub = subs[pos]
        if test_row.loc[pos] != sub:
            test_row.loc[pos] = sub
def convert_sub_str(sub_str):
    #Converts a substitution string (i.e. P39T) into a tuple of (position,outgroup variant,test variant) 
    #for example 'P39T' -> (39,'P','T')
    try:
        groups = re.search("([A-Z])(\d+)([A-Z])",sub_str).groups()
        out,pos,test = groups[0],int(groups[1]),groups[2]
        return (pos,out,test)
    except AttributeError as e:
#         raise UserWarning("oops")
        print(sub_str+" did not match expected pattern i.e. P39T, A79V")
        print("Will not attempt to realign for this input string")
        return None

    

def force_with_shifts(align_df,test_species,sub_str,gap_dict,unique_gaps):
#subs is a dictionary from positions to tuples of (outgroup substitution, test variant)
    sub = convert_sub_str(sub_str)
    if sub:
        seq_pos,out,test = sub[0],sub[1],sub[2]
        #Check test is at seq_pos
        align_pos = seq_to_align_pos(seq_pos,test_species,gap_dict)
        test_variant = align_df.loc[test_species,align_pos]
        if test_variant != test:
            print("Input string: "+sub_str)
            print("Could not find specified substitution at specified position. Alignment will not be changed Continuing...")
            pass
        else:
            #Idea: generate outgroup seq pos for each outgroup species individually. Check consensus of position, use mode to select for predicted position. Check substitution at that position. Threshold. 
            outgroup = align_df.index.copy().drop(test_species)
            for out_species in outgroup:
                out_seq_pos = outgroup_seq_pos(gap_dict,seq_pos,test_species,out_species)
                out_align_pos = seq_to_align_pos(out_seq_pos,out_species,gap_dict)
    else:    
        pass
        
def align_pos_to_native_pos(align_df, test_species, align_pos):
    pre_pos = align_df.loc[test_species,:align_pos-1]
    pre_pos_vc = pre_pos.value_counts()
    native_pos = align_pos
    if '-' in pre_pos_vc:
        native_pos -= pre_pos_vc['-']
    return native_pos
# gap_dict, unique_gaps, unique_subs = species_gaps_dict(align_df)
# force_with_shifts(align_df,test_species,"P39T",gap_dict,unique_gaps)
# force_with_shifts(align_df,test_species,"R51R",gap_dict,unique_gaps)
# subs = {39:("P","T")}
# align_row = align_df.loc[test_species]
# print(seq_to_align_pos(15,test_species,gap_dict)) #Example tests

In [22]:
# pd.options.display.max_columns = 999
# display(final_align_df.loc[:,:50])
# npos = align_pos_to_native_pos(final_align_df,test_species,36)
# print(npos)


In [23]:
#Returns a range of sliding window positions to consider. All sliding windows of length n in this range should #
#include pos and not produce IndexErrors. If there is a gap at pos, then the sliding window extends one more in each direction
#1-indexed to match generated dataframes 
def sw_list(pos,align_len,gaps,n=7):
    if pos-n < 1:
        sw = list(range(1,pos+n))
    elif pos+n > align_len:
        sw = list(range(pos-n+1,align_len+1))
    else:
        sw = list(range(pos-n+1,pos+n))
    for gap in gaps: 
        if gap in sw:
            first,last = sw[0],sw[-1]
            if gap < pos and first > 1:
                sw.remove(gap)
                sw.insert(0,first-1)
            elif gap == pos:
                pass
                #TODO
            else:
                sw.remove(gap)
                if last+1 <= align_len:
                    sw.append(last+1)
                else:
#                     print("boop")
                    sw.insert(0,first-1)
#                 print(sw_)
    pos_idx = sw.index(pos)
    if pos_idx > 6:
        return sw[pos_idx-n+1:]
    elif len(sw) - pos_idx > 6:
        return sw[:pos_idx+7]
    else:
        return sw

def drop_gaps(score_series,gaps):
    copy = score_series.copy()
    for gap in gaps:
        if gap in copy.index:
            copy.drop(gap,inplace=True)
    return copy
#Returns both the 1-indexed list of positions of max score sliding window and the mean score for that sliding window 
#Ignore gap columns from mean score
def max_sw(score_series, sw_, gaps, n=7):
    sw_series = score_series[sw_]
    sw_series = drop_gaps(sw_series,gaps)
    sw_nd = sw_series.values
    size = len(sw_nd)-n+1
    sum_arr = []
    if size > 0:
        for i in range(size):
            sum_arr.append(np.mean(sw_nd[i:i+n]))
        max_nd_idx = np.argmax(sum_arr)
        avg_score = sum_arr[max_nd_idx]
        window = sw_series.index[max_nd_idx:max_nd_idx+n] 
    else: 
        avg_score = np.mean(sw_nd)
        window = sw_series.index
    return window,avg_score
            

In [24]:
#Test Cell for sliding window ranges and max score checking 
# test_srs = pd.Series([0,0,2,1,1,3,2,1,2,1,1,0,0,0,0,0,0,0,1,2,1,1,1,],index=range(1,24))
# swr = sw_list(5,len(test_srs),[])
# print(swr)
# gap_positions = [12,13]
# max_sw(test_srs,swr,gap_positions)

In [25]:
#vectorized seq weights
#Calculates Henikoff (1994) sequence weights form msa
from collections import Counter 
def calculate_sequence_weights(msa,aas):
    """ Calculate the sequence weights using the Henikoff '94 method
    for the given msa. """
    n = len(msa)
    alignlen = len(msa[0])
    seq_weights = np.zeros((n))
    for col in msa.T:
        freqs = Counter(col)
        del freqs['-']
        num_observed = len(freqs)
        
#         d_ = [freqs[aa]*num_observed if aa != '-' else 0 for aa in col]
        d_ = [freqs[aa]*num_observed for aa in col]
        inverse = [1/d if d>0 else 0 for d in d_]
        seq_weights += inverse
    seq_weights /= alignlen
    return seq_weights
# seq_weights = calculate_sequence_weights(align_nd,aas)

#Test works yay
# test = np.array([['G','Y','V','G','S'],['G','F','D','G','F'],['G','Y','D','G','F'],['G','Y','Q','G','G']])
# calculate_sequence_weights(test,aas)
#Checked against values and examples provided in the Henikoff paper

In [26]:
#Generates Pc distribution for column c. Distribution is weighted 
def weighted_fcpseudo(col, seq_weights, pc_amount=0.000001):
    if len(seq_weights) != len(col):
        return [1]*len(col)
    else:
        freqs = np.array([pc_amount]*len(aas))
        for i,aa in enumerate(aas):
#             print(col)
#             print([seq_weights[j] if aa == entry else 0 for j,entry in enumerate(col)])
            freqs[i] += sum([seq_weights[j] if aa == entry else 0 for j,entry in enumerate(col)])
        for i,entry in enumerate(freqs): 
#             if entry > pc_amount:
#                  freqs[i] -= pc_amount
            freqs[i] /= (sum(seq_weights) + len(aas)*pc_amount)
        return freqs
# test_col = ["G","Y","T","-"]
# test_col = ["M","M","M","M"]
# weights = calculate_sequence_weights(test,aas)
# test_ = weighted_fcpseudo(test_col,weights)

In [27]:

def weighted_gap_penalty(col,weights):
    #Return simple gap penalty multiplier for column 
    gap_weights = np.array([w if c == "-" else 0 for c,w in zip(col,weights)])
    gap_sum = sum(gap_weights)
    return 1 - (gap_sum/sum(weights))
# print(weighted_gap_penalty(['A','A','-'],[0.3333,0.3333,0.3333]))
def relative_entropy(pC,bg,gap_penalty=0):
    """Calculate the relative entropy of the column distribution with a 
    background distribution specified in bg_distr. This is similar to the 
    approach proposed in Wang and Samudrala 06."""
    if len(bg) == 20 and len(pC) == 21:
        #Remove gap count
        pC = pC[:-1]
        pC = pC/(sum(pC))
    ratio = pC/bg
    log_ratio = np.log(ratio)/np.log(2)
    RE = sum([a*b for a,b in zip(pC,log_ratio)])
    if gap_penalty:
        RE*= gap_penalty
    return RE

def col_RE(col,bg,weights,gap_penalty=1):
    pC = weighted_fcpseudo(col,weights)
    if len(bg) == 20 and len(pC) == 21:
        pC = pC[:-1]
        pC /= sum(pC)
    ratio = pC/bg
    log_ratio = np.log(ratio)/np.log(2)
    RE = sum([a*b for a,b in zip(pC,log_ratio)])
    if gap_penalty:
        RE*= gap_penalty
    return RE
# relative_entropy(np.array([0.25,0.25,0.25,0.25]),np.array([0.1,0.1,0.1,0.7]))

def JSD(col,bg,weights,gap_penalty=1,jsd_lam=0.5):
    pC = weighted_fcpseudo(col,weights)
    if len(bg) == 20:
        #Remove gap count
        pC = pC[:-1]
        pC = pC/(sum(pC))
    gap_penalty = weighted_gap_penalty(col,weights)
    r_dist = np.array([jsd_lam*aa for aa in pC]) + np.array([(1-jsd_lam)*aa for aa in bg])

    RE_pCr = relative_entropy(pC,r_dist,gap_penalty)
    RE_qr = relative_entropy(bg,r_dist, gap_penalty)
    jsd = jsd_lam*RE_pCr + (1-jsd_lam)*RE_qr
    return jsd 

In [28]:
def calc_z_scores(scores):
#     if type(scores) == pd.Series:
#         scores = scores.values
    mean = np.mean(scores)
    std = np.std(scores)
    z_scores = (scores-mean)/std
    return z_scores

In [29]:
#Generate a Jensen Shannon Difference Series for position indices within the MSA
def generate_jsd_series(test_species,align_df, alignlen, aas,blosum62_bg):
    jsd_srs = pd.Series(index=range(1,alignlen))
    jsd_lam = 0.5
    gap_threshold = 0.3
    gap_positions = []
    jsd_df = align_df.drop(test_species,axis=0)
    # jsd_df = align_df.copy()
    jsd_nd = jsd_df.values
    weights = calculate_sequence_weights(jsd_nd,aas)
    # print(weights)
    for i,col in enumerate(jsd_nd.T):
        gap_penalty = weighted_gap_penalty(col,weights)
        jsd = JSD(col,blosum62_bg,weights,gap_penalty)
        jsd_srs[i+1] = jsd
        if gap_ratio(col) > gap_threshold:
            gap_positions.append(i+1)
    jsd_zscores = calc_z_scores(jsd_srs)
    with pd.option_context("display.max_rows",None,"display.max_columns", None):
#         display(jsd_srs)
        pass
    return jsd_srs, jsd_zscores, gap_positions
    #     display(z_scores)
    #     display(jsd_df)
#     print(gap_positions)
# jsd_srs, jsd_zscores, gap_positions = generate_jsd_series(align_df, alignlen, aas,blosum62_bg)

In [30]:
#Sliding window calculations
def calc_sw_metrics(score_srs, uniques, gap_positions, alignlen, config,n=7):
    mean = np.mean(score_srs)
    std = np.std(score_srs)
    display_windows = config.getboolean("SW_verbose")

#     print("Sliding Window Length: "+str(n))
#     print("Score Mean: "+str(mean))
#     print("Score Standard Deviation: "+str(std))
    unique_positions = uniques.columns
    pos_jsds = score_srs.loc[unique_positions]
    pos_jsd_zs = (pos_jsds-mean)/std
    sw_jsds = pd.Series(index=unique_positions,name="Sliding Window Max Mean")
    sw_zs = pd.Series(index=unique_positions,name="Sliding Window Z Score")
    sw_indices = pd.Series(index=unique_positions,name="Start",dtype=int)
    for sub in uniques:
#         sw_ = sw_list(sub,alignlen,gap_positions,n)
#         window, window_score = max_sw(score_srs,sw_,gap_positions)
#         sw_indices[sub] = str(list(window))
#         sw_jsds[sub] = window_score
#         sw_df = align_df.loc[:,window]
#         window_z = (window_score-mean)/std
#         sw_zs[sub] = window_z
#         pos_jsd = jsd_srs[sub]
#         pos_z = (pos_jsd-mean)/std
#         if display_windows:
#             print("Position: "+str(sub))
#             print("Max Score Sliding Window: ")
#             display(sw_df)
#             print("Mean JSD for window: "+str(window_score))
#             print("Z score of mean JSD: "+str(window_z))
#             print("Position JSD: " + str(pos_jsd))
#             print("Z score of position JSD: "+str(pos_z))
#             print("===============================================")
#     metrics = (pos_jsds, pos_jsd_zs, sw_scores, sw_zscores, sw_indices)
        pass
    data_dict = {"JSD":pos_jsds, "JSD Z-Score": pos_jsd_zs,"Window JSD": sw_jsds, "Window JSD Z-Score":sw_zs, "Window":sw_indices}
    metrics = pd.DataFrame(index = unique_positions,data=data_dict)
    return metrics 
# sw_metrics = calc_sw_metrics(jsd_srs, uniques, gap_positions,alignlen, config)
# pos_jsds, pos_jsd_zs, sw_scores, sw_zscores, sw_indices = sw_metrics
# print(pos_jsds)

In [57]:
#Summary table
def summary_table(align_df, sw_metrics,test_species,blos_df, display_summary):
    """
    """
#     display_summary = config.getboolean("Summary_verbose")
    pos_jsds, pos_jsd_zs, sw_scores, sw_zscores, sw_indices = sw_metrics
    jsd_nd = align_df.copy().drop(index=test_species).values
#     unique_positions = uniques.columns
    unique_positions = sw_metrics.columns
    align_nd = align_df.values
    gap_ratios, test_variants, sub_freq = pd.Series(index=unique_positions), pd.Series(index=unique_positions), \
                 pd.Series(index=unique_positions,dtype=int)
#     test_variants = pd.Series(index=unique_positions)
#     sub_freq = pd.Series(index=unique_positions,dtype=int)
    blosum62_col = pd.Series(index=unique_positions,dtype=np.float64)
    blosum62_pairwise = pd.Series(index=unique_positions)
    for pos in unique_positions:
        pos_jsds, pos_jsdzs = sw_metrics.loc[:,"JSD"], sw_metrics.loc[:,"JSD Z-Score"]
        align_col = align_nd.T[pos-1]
        align_col_srs = align_df.loc[:,pos]
        n_seq = len(align_col_srs)
        gap_ratios[pos] = gap_ratio(align_col)
        test_species_variant = align_df.loc[test_species,pos]
        test_variants[pos] = test_species_variant
        vc = align_df[pos].value_counts()
        sub_freq[pos] = vc[test_species_variant]
        col = jsd_nd.T[pos-1]
        if not test_species_variant == '-':
            blos_sum = np.mean(blos_df[test_species_variant][col])
            blosum62_col[pos] = blos_sum
        else:
            blosum62_col[pos] = np.nan
        outgrp_vars, outgrp_counts = np.unique(col,return_counts=True)
        
        #pairwise calculations
        others_col = jsd_nd.T[pos-1]
        pairwise_scores = []
        skip_chars = ['-','X']
        for i,first in enumerate(others_col):
            for j,second in enumerate(others_col):
                if i<j:
                    if not first in skip_chars and not second in skip_chars:
                        pairwise_scores.append(blos_df[first][second])
                    else: 
                        pass
        if pairwise_scores:
            blosum62_pairwise[pos] = np.mean(pairwise_scores)
        else:
            blosum62_pairwise[pos] = np.nan

    data = {"Test Variant":test_variants,"Variant Instances":sub_freq,"Window Positions":sw_indices, \
            "Gap Fraction":gap_ratios, "JSD":pos_jsds,"JSD Z-Score":pos_jsd_zs,\
            "Window Z-Score":sw_zscores, "Test vs Outgroup Blosum62":blosum62_col, \
            "Outgroup Pairwise Blosum62":blosum62_pairwise}
    summary_df = pd.DataFrame(index=unique_positions,data=data)
    filterthreshold = 0.4
    filtered = summary_df[summary_df["Window Z-Score"] >= filterthreshold]
    #Output conditional on config option
    if display_summary:
        print(test_species+" Semi-Unique Substitutions and Sliding Window Scores")
        print("Number of Species: "+str(len(align_df.index)))
        print("Positions with gap percentages over 0.3 were omitted from sliding window calculations")
        with pd.option_context("display.max_rows",None,"display.max_columns", None,"display.max_colwidth",200):
            display(summary_df)
        
        print("Filtered to Window Z-Score Threshold of "+str(filterthreshold))
        with pd.option_context("display.max_rows",None,"display.max_columns", None,"display.max_colwidth",200):
            display(filtered)
#             pass
    return summary_df
# display_summary = config.getboolean("Summary_verbose")
# summary_df = summary_table(align_df,uniques,sw_metrics,test_species,blos_df,display_summary)

def summary_table2(align_df, sw_metrics,test_species,blos_df, display_summary):
#     pos_jsds, pos_jsd_zs, sw_scores, sw_zscores, sw_indices = sw_metrics
    pos_jsds, pos_jsdzs = sw_metrics.loc[:,"JSD"], sw_metrics.loc[:,"JSD Z-Score"]
    jsd_nd = align_df.copy().drop(index=test_species).values
    unique_positions = sw_metrics.index
    unique_positions.name = "MSA Position"
    n_seq = len(align_df.index)
    cols_list = ["Test Species Position", "Test Variant","Test Variant Instances","Outgroup Variant",\
                 "Outgroup Variant Instances","Aligned Sequences", "Gap Fraction", "JSD","JSD Z-Score", \
                 "Test vs Outgroup Blosum62", "Outgroup Pairwise Blosum62"]
    summary_df = pd.DataFrame(index=unique_positions,columns=cols_list)
    for pos in unique_positions:
        native_pos = align_pos_to_native_pos(align_df, test_species,pos)
        jsd = pos_jsds[pos]
        jsd_z = pos_jsdzs[pos]
        align_col_srs = align_df.loc[:,pos]
        align_col = align_col_srs.values
        outgroup_variant = align_col_srs.mode().iloc[0]
        gap_fraction = gap_ratio(align_col)
        test_variant = align_df.loc[test_species,pos]
        vc = align_df[pos].value_counts()
        og_var_freq = vc[outgroup_variant]
        test_var_freq = vc[test_variant]
        col = jsd_nd.T[pos-1]
        if not test_variant == '-':
            blos_mean = np.mean(blos_df[test_variant][col])
        else:
            blos_mean = np.nan
        #pairwise calculations
        others_col = jsd_nd.T[pos-1]
        pairwise_scores = []
        skip_chars = ['-','X']
        for i,first in enumerate(others_col):
            for j,second in enumerate(others_col):
                if i<j:
                    if not first in skip_chars and not second in skip_chars:
                        pairwise_scores.append(blos_df[first][second])
                    else: 
                        pass
        if pairwise_scores:
#             blosum62_pairwise[pos] = np.mean(pairwise_scores)
            blos_pw = np.mean(pairwise_scores)
        else:
            blos_pw = np.nan
        row_values = [native_pos, test_variant, test_var_freq,outgroup_variant,og_var_freq,n_seq,gap_fraction,jsd,jsd_z,blos_mean,blos_pw]
        summary_row = pd.Series(name=pos,data=dict(zip(cols_list,row_values)))
#         summary_df = summary_df.append(summary_row)
        summary_df.loc[pos] = summary_row
    #Output conditional on config option
    if display_summary:
        print(test_species+" Semi-Unique Substitutions Summary")
        print("Number of Species: "+str(len(align_df.index)))
        with pd.option_context("display.max_rows",None,"display.max_columns", None,"display.max_colwidth",200):
            display(summary_df)
#             pass
    return summary_df

In [32]:
# #Display Alignment Table and Unique subs for Reference
# with pd.option_context("display.max_rows",None,"display.max_columns", None):
# #     display(align_df)
# #     display(uniques)
# #     display(align_df)
#     pass

In [65]:
def read_summary_output(summary_path, gene_name, window_calculations=False, filter_output=False):
    #Reads summary data from previous run from csv file
    #Filtering optional
    #Formats so that can be readily appended to overall_summary (reindexes, includes gene name)
    filter_output = False
    summary_df = pd.read_csv(summary_path,index_col=0)
    summary_df.reset_index(drop=False,inplace=True)
    summary_df.rename(columns={"index":"Position"},inplace=True)
    if "Gene" not in summary_df.columns:
        summary_df.insert(0,"Gene",[gene_name]*len(summary_df))
    if filter_output:    
        filtered = filter_summary(gene_name,summary_df, window_calculations)
        if len(filtered) < 1:
            print("No results above Z-threshold/ gap fraction threshold")
        else:
            print("Filtered to Z-threshold = 0.6, or gap fraction > 0.85:")
            return filtered
    else:
        return summary_df
def write_output(gene_name,run_name,MSA_outfile,summary_df):
    #Writes out MSA as a clustal format 
    create_directory(run_name+"/output/"+gene_name)
    MSA_path = run_name+"/output/"+gene_name+"/"+gene_name+".fasta"
#     records_generator = SeqIO.parse(open(MSA_outfile),"fasta")
    SeqIO.convert(MSA_outfile,"fasta",MSA_path,"fasta")
    summary_path = run_name+"/output/"+gene_name+"/"+gene_name+"_summary.csv"
    summary_df.to_csv(summary_path)
    
def write_output2(gene_name,run_name,records_df,summary_df=pd.DataFrame()):
    align_srs = records_df["align"]
    gene_dir_path = run_name+"/output/{0}".format(gene_name)
    create_directory(gene_dir_path)
    align_path = gene_dir_path+"/{0}".format(gene_name)+"_MSA.fasta"
    std_path = gene_dir_path+"/{0}".format(gene_name)+".fasta"
    srs_to_fasta(align_srs,align_path)
    align_srs_to_seq_srs(align_srs, outfile_path=std_path)
    drop_align = records_df.drop(columns="align",inplace=False)
    records_path = gene_dir_path+"/{0}".format(gene_name)+"_records.csv"
    drop_align.to_csv(records_path,float_format='%.5f')
    if not summary_df.empty:
        summary_path = gene_dir_path+"/{0}".format(gene_name)+"_summary.csv"
        summary_df.to_csv(summary_path,float_format='%.5f')
        
def write_output3(gene_name,run_name,records_df,summary_df=pd.DataFrame()):
    """.fasta file output writing (both pre and post alignment) are now handled during 
    process_input (specifically in seq_srs_to_align_df by calling construct_id_dm)"""
    gene_dir_path = "{0}/output/{1}".format(run_name,gene_name)
#     create_directory(gene_dir_path)
    drop_align = records_df.drop(columns="seq",inplace=False)
#     display(drop_align)
    records_path = "{0}/{1}_records.csv".format(gene_dir_path,gene_name)
    drop_align.to_csv(records_path,float_format='%.5f')
    if not summary_df.empty:
        summary_path = "{0}/{1}_summary.csv".format(gene_dir_path,gene_name)
        summary_df.to_csv(summary_path,float_format='%.5f')
        
def write_output4(records_fpath,summary_fpath,records_df=pd.DataFrame(),summary_df=pd.DataFrame(),write_seq=False):
    """.fasta file output writing (both pre and post alignment) are now handled during 
    process_input (specifically in seq_srs_to_align_df by calling construct_id_dm)"""
    if not records_df.empty:
        if not write_seq and "seq" in records_df:
            drop_seq = records_df.drop(columns="seq",inplace=False)
            drop_seq.to_csv(records_fpath,float_format='%.5f')
        else:
            records_df.to_csv(records_fpath,float_format='%.5f')
    if not summary_df.empty:
#         summary_path = "{0}/{1}_summary.csv".format(gene_dir_path,gene_name)
        summary_df.to_csv(summary_fpath,float_format='%.5f')        
        

def filter_summary(gene_name, summary_df, window_calculations=True,jsd_thresh=0.6, gap_thresh=0.85):
    overall_add = summary_df.copy()
    filtered = overall_add[overall_add["JSD Z-Score"] > jsd_thresh]
    filtered = filtered.append(overall_add.loc[(overall_add["Gap Fraction"] > gap_thresh) & ~(overall_add["Test Variant"] == '-')])
#     filtered.reset_index(drop=False,inplace=True)
    return filtered

In [35]:
#Old input schema - 
# #Input parsing and background distribution information
# DEFAULT_SPECIES_PATH = "config/v10_0_species.txt"
# aas, blosum62_bg, blos_df, sim_matrix = gen_blos_df()
# config = parse_config()
# run_name = config["RunName"]
# create_run_directory(run_name)
# spec_list, spec_hc = parse_species(DEFAULT_SPECIES_PATH)
# write_run_params_file(config, DEFAULT_SPECIES_PATH, spec_hc)
# genes_file = config["GenesFile"]
# gene_list = parse_genes(genes_file)
# tax_table = odb_tablev10(spec_list)

# UNIQUE_THRESH = 0.1
# DROP_SPEC_LIST = ["10141_0","13616_0","9258_0","9483_0","9544_0","9598_0","9685_0","43346_0","246437_0","9925_0",\
#                  "9940_0","9646_0","74533_0","9365_0","9785_0","34839_0"]

# #Config Settings invariant of gene; creation of subfolders for run
# test_species_name = config["TestSpecies"]
# display_MSA = config.getboolean("MSA_verbose")
# display_uniques = config.getboolean("Uniques_verbose")
# display_summary = config.getboolean("Summary_Verbose")
# errors_fpath = '{0}/summary/errors.tsv'.format(run_name)
# seq_qc_fpath = '{0}/summary/seq_QC.tsv'.format(run_name)
# if os.path.exists(errors_fpath):
#     check_errors = True
#     errors_df = pd.read_csv(errors_fpath,delimiter="\t",index_col="gene")
#     error_genes = errors_df.index
# else:
#     check_errors = False
# #Acquire input
# valid_searches, failed_searches = download_input_data(gene_list,tax_table,config) 
# #Run analysis for each gene in genes list
# window_calculations = False
# OVERWRITE_ANALYSIS = False
# if window_calculations:
#     summary_cols = ["Gene","Position","Test Variant","Test Variant Instances","Window Positions", "Gap Fraction",\
#                     "JSD","JSD Z-Score","Window Z-Score", \
#                     "Test vs Outgroup Blosum62", "Outgroup Pairwise Blosum62"]
# else:
#     summary_cols = ["Gene","Test Species Position","MSA Position","Test Variant","Test Variant Instances",\
#                     "Outgroup Variant", "Aligned Sequences", "Gap Fraction", "JSD","JSD Z-Score", \
#                      "Test vs Outgroup Blosum62", "Outgroup Pairwise Blosum62"]
# numeric_cols = ["Gap Fraction","JSD","JSD Z-Score", \
#                 "Test vs Outgroup Blosum62", "Outgroup Pairwise Blosum62"]
# overall_summary = pd.DataFrame(columns=summary_cols)
# # for col in numeric_cols:
# #     overall_summary[col] = overall_summary[col].astype(np.float64)
# for gene_name in valid_searches[:]:
# # for gene_name in ["ATPIF1","ATP5A1","COX6B1"]:
#     empty_directory("tmp")
#     print("========="+gene_name+"=========")
#     summary_path = run_name+"/output/"+gene_name+"/"+gene_name+"_summary.csv"
#     if check_errors and gene_name in error_genes:#error_genes.str.contains(gene_name).any():
#         error_row = errors_df.loc[gene_name,:]
#         error_type, error_code, error_str = error_row.values
#         print("{0}\t{1}\t{2}\t{3}".format(gene_name,error_type,error_code,error_str))
#     elif os.path.exists(summary_path) and not OVERWRITE_ANALYSIS:
#         summary_df = read_summary_output(summary_path,gene_name,window_calculations)
#         overall_summary = overall_summary.append(summary_df,sort=False)
#     else:
#         try:
#             final_records_df, final_align_df = process_input(gene_name,run_name,test_species,errors_fpath,seq_qc_fpath,drop_spec_list=DROP_SPEC_LIST)            
#             test_species_record = final_records_df.loc[final_records_df["organism_name"]==test_species_name,:]
#             test_species_id = test_species_record.index[0]
#         except (OrthoDBQueryError, SequenceDataError) as e:
#             write_errors(errors_fpath,gene_name,e)
#             continue
#         n, align_len = len(final_align_df.index), len(final_align_df.columns)
#         #Unique substitutions must be present in <20% of sequences
#         sub_freq_threshold = max(int(n*UNIQUE_THRESH), 1)
#         uniques = find_uniques(final_align_df,sub_freq_threshold,test_species_id,display_uniques)
#         if len(uniques.columns) == 0:
#             SA_error = SequenceAnalysisError(0,"No species unique substitutions under occurence threshold ({0}%)".format(100*UNIQUE_THRESH))
#             write_errors(errors_fpath,gene_name,SA_error)
#             write_output3(gene_name,run_name,final_records_df)
#             continue
#         else:
#             align_nd = final_align_df.values
#             seq_weights = calculate_sequence_weights(align_nd,aas)
#             #Unique substitution identification, metrics calculations and individual gene output writing
#             gap_dict, unique_gaps, unique_subs = species_gaps_dict(final_align_df)
#             jsd_srs, jsd_zscores, gap_positions = generate_jsd_series(test_species_id,final_align_df,align_len,aas,blosum62_bg)
#             sw_metrics = calc_sw_metrics(jsd_srs, uniques, gap_positions,align_len, config)
#             if window_calculations:
#                 summary_df = summary_table(final_align_df,sw_metrics,test_species_id,blos_df,display_summary)
#             else:
#                 summary_df = summary_table2(final_align_df,sw_metrics,test_species_id,blos_df,display_summary)
#             write_output3(gene_name,run_name,final_records_df,summary_df)
#             summary_df.reset_index(drop=False,inplace=True)
#             if not "Gene" in summary_df.columns:
#                 summary_df.insert(0,"Gene",[gene_name]*len(summary_df))
#             overall_summary = overall_summary.append(summary_df,sort=False)
# overall_summary.reset_index(drop=True,inplace=True)
# overall_summary = overall_summary[summary_cols]
# display(overall_summary)
# # if window_calculations:
# #     ordered_cols = ["Gene","Position","Test Variant","Variant Instances","Gap Fraction","Position JSD",\
# #                     "Position JSD Z-Score","Test vs Outgroup Blosum62","Outgroup Pairwise Blosum62",\
# #                     "Window Positions","Window Z-Score"]
# # else:
# #     ordered_cols = ["Gene","Position","Test Variant","Variant Instances","Gap Fraction","Position JSD",\
# #                     "Position JSD Z-Score","Test vs Outgroup Blosum62","Outgroup Pairwise Blosum62",\
# #                     "Window Positions","Window Z-Score"]
# # overall_summary = overall_summary[ordered_cols]
# # overall_summary.rename(columns={"index":"Position"},inplace=True)
# # display(overall_summary)
# overall_summary_path = run_name+"/summary/summary.csv"
# overall_summary.to_csv(overall_summary_path,float_format='%.5f')
# plt.scatter(x=overall_summary["JSD"],y=overall_summary["Test vs Outgroup Blosum62"],alpha=0.2)
# plt.xlabel("Position JSD")
# plt.ylabel("Average Test Species vs Outgroup Blosum62")
# plt.title("Distribution of JSD/ Blosum for all input unique subs")
# plt.savefig(run_name+"/summary/summary_scatter.png")

In [84]:
#Modified input schema - use pre-aligned AGS to 13LGS sequences

# #Input parsing and background distribution information
DEFAULT_SPECIES_PATH = "config/v10_0_species.txt"
aas, blosum62_bg, blos_df, sim_matrix = gen_blos_df()
config = parse_config()
run_name = config["RunName"]
create_run_directory(run_name)
# spec_list, spec_hc = parse_species(DEFAULT_SPECIES_PATH)
# write_run_params_file(config, DEFAULT_SPECIES_PATH, spec_hc)
genes_file = config["GenesFile"]
gene_list = parse_genes(genes_file)
tax_table = odb_tablev10(spec_list)

UNIQUE_THRESH = 0.1
# DROP_SPEC_LIST = ["10141_0","13616_0","9258_0","9483_0","9544_0","9598_0","9685_0","43346_0","246437_0","9925_0",\
#                  "9940_0","9646_0","74533_0","9365_0","9785_0","34839_0"]

# #Config Settings invariant of gene; creation of subfolders for run
test_species_name = config["TestSpecies"]
errors_fpath = '{0}/summary/errors.tsv'.format(run_name)
seq_qc_fpath = '{0}/summary/seq_QC.tsv'.format(run_name)
AGS_aln_base_dir = "{0}/allspec_AGS_alignments".format(run_name)
output_base_dir = "{0}/output".format(run_name)

if os.path.exists(errors_fpath):
    check_errors = True
    errors_df = pd.read_csv(errors_fpath,delimiter="\t",index_col="gene")
    errors_df = errors_df.loc[~(errors_df["error_type"]=="SequenceAnalysisError"),:]
    error_genes = errors_df.index
    display(errors_df)
else:
    check_errors = False
# #Acquire input
# valid_searches, failed_searches = download_input_data(gene_list,tax_table,config) 
# #Run analysis for each gene in genes list
# window_calculations = False
OVERWRITE_ANALYSIS = False
overall_summary_cols = ["Gene","MSA Position","Test Species Position","Test Variant","Test Variant Instances", \
                "AGS Variant", "AGS 13LGS Consensus","Outgroup Variant", "Outgroup Variant Instances",\
                "Aligned Sequences", "Gap Fraction", "JSD","JSD Z-Score", \
                 "Test vs Outgroup Blosum62", "Outgroup Pairwise Blosum62"]
updated_summary_ordered_cols = ["Test Species Position","Test Variant","Test Variant Instances", \
                                "AGS Variant", "AGS 13LGS Consensus","Outgroup Variant", "Outgroup Variant Instances",\
                                "Aligned Sequences", "Gap Fraction", "JSD","JSD Z-Score", \
                                 "Test vs Outgroup Blosum62", "Outgroup Pairwise Blosum62"]
numeric_cols = ["Gap Fraction","JSD","JSD Z-Score", \
                "Test vs Outgroup Blosum62", "Outgroup Pairwise Blosum62"]
# dtype_specification = zip(numeric_cols,[np.float64]*len(numeric_cols))
overall_summary = pd.DataFrame(columns=overall_summary_cols)
for col in numeric_cols:
    overall_summary[col] = overall_summary[col].astype(np.float64)
no_uniques = []
for gene_name in gene_list[:]:
    empty_directory("tmp")
    print("========="+gene_name+"=========")
    
    AGS_aln_gene_dir = "{0}/{1}".format(AGS_aln_base_dir,gene_name)
    output_gene_dir = "{0}/{1}".format(AGS_aln_base_dir,gene_name)
    AGS_MSA_fpath = "{0}/{1}_MSA.fasta".format(AGS_aln_gene_dir,gene_name)
    
    all_records_fpath = "{0}/{1}_records.csv".format(AGS_aln_gene_dir,gene_name)
    prev_summary_fpath = "{0}/{1}_summary.csv".format(AGS_aln_gene_dir,gene_name)
    updated_summary_fpath = "{0}/{1}_AGS_summary.csv".format(AGS_aln_gene_dir,gene_name)
    
    if check_errors and gene_name in error_genes:#error_genes.str.contains(gene_name).any():
        error_row = errors_df.loc[gene_name,:]
        error_type, error_code, error_str = error_row.values
        print("{0}\t{1}\t{2}\t{3}".format(gene_name,error_type,error_code,error_str))
    elif not os.path.exists(AGS_MSA_fpath):
        AGS_SD_error = SequenceDataError(6,"No AGS NCBI sequences available.")
        write_errors(errors_fpath,gene_name,AGS_SD_error)
        continue
    elif os.path.exists(updated_summary_fpath) and not OVERWRITE_ANALYSIS:
        summary_df = read_summary_output(updated_summary_fpath,gene_name)
        overall_summary = overall_summary.append(summary_df,sort=False)
    else:
        AGS_MSA_srs = fasta_to_srs(AGS_MSA_fpath)
        AGS_MSA_df = align_srs_to_df(AGS_MSA_srs)
        all_records_df = pd.read_csv(all_records_fpath,index_col="Unnamed: 0")
        
        AGS_idx = AGS_MSA_df.index[AGS_MSA_df.index.str.contains('XP_')][0]
        LGS_idx = AGS_MSA_df.index[AGS_MSA_df.index.str.contains('43179_')][0]
        AGS_row = AGS_MSA_df.loc[AGS_idx,:]
        ODB_MSA_df = AGS_MSA_df.drop(index=AGS_idx)
        n, align_len = len(ODB_MSA_df.index), len(ODB_MSA_df.columns)
#         #Unique substitutions must be present in < UNIQUE_THRESH*100 % of sequences (or min of 1) 
        sub_freq_threshold = max(int(n*UNIQUE_THRESH), 1)
        uniques = find_uniques(ODB_MSA_df,sub_freq_threshold,LGS_idx,display_uniques=False)
        if len(uniques.columns) == 0:
#             write_output3(gene_name,run_name,final_records_df)
            no_uniques.append(gene_name)
            continue
        else:
            align_nd = ODB_MSA_df.values
            seq_weights = calculate_sequence_weights(align_nd,aas)
            #Unique substitution identification, metrics calculations and individual gene output writing
            gap_dict, unique_gaps, unique_subs = species_gaps_dict(ODB_MSA_df)
            jsd_srs, jsd_zscores, gap_positions = generate_jsd_series(LGS_idx,ODB_MSA_df,align_len,aas,blosum62_bg)
            sw_metrics = calc_sw_metrics(jsd_srs, uniques, gap_positions,align_len, config)
#             if window_calculations:
#                 summary_df = summary_table(final_align_df,sw_metrics,test_species_id,blos_df,display_summary)
#             else:
            summary_df = summary_table2(ODB_MSA_df,sw_metrics,LGS_idx,blos_df,display_summary=False)
            write_output4(all_records_fpath,prev_summary_fpath,summary_df=summary_df)

            MSA_pos_idx = summary_df.index
            AGS_var_srs = AGS_row.loc[MSA_pos_idx]
            AGS_13LGS_consensus_srs = AGS_var_srs==summary_df["Test Variant"]
            updated_summary_df = summary_df.copy()
            updated_summary_df.loc[:,"AGS Variant"] = AGS_var_srs
            updated_summary_df.loc[:,"AGS 13LGS Consensus"] = AGS_13LGS_consensus_srs
            
            updated_summary_df = updated_summary_df[updated_summary_ordered_cols]
            write_output4(all_records_fpath,updated_summary_fpath,summary_df=updated_summary_df)
            updated_summary_df.reset_index(drop=False,inplace=True)
            if not "Gene" in updated_summary_df.columns:
                updated_summary_df.insert(0,"Gene",[gene_name]*len(updated_summary_df))
            overall_summary = overall_summary.append(updated_summary_df,sort=False)

display(overall_summary)
overall_summary_path = run_name+"/summary/summary.csv"
overall_summary.to_csv(overall_summary_path,float_format='%.5f')
# plt.scatter(x=overall_summary["JSD"],y=overall_summary["Test vs Outgroup Blosum62"],alpha=0.2)
# plt.xlabel("Position JSD")
# plt.ylabel("Average Test Species vs Outgroup Blosum62")
# plt.title("Distribution of JSD/ Blosum for all input unique subs")
# plt.savefig(run_name+"/summary/summary_scatter.png")

,error_type,error_code,error_str
gene,,,
CDC42,OrthoDBQueryError,1,OrthoDB search yielded too many clusters
GABP1,OrthoDBQueryError,0,No OrthoDB results for query
HSF2B,OrthoDBQueryError,0,No OrthoDB results for query
OLFR710A,OrthoDBQueryError,0,No OrthoDB results for query
PDIA3P1,OrthoDBQueryError,0,No OrthoDB results for query
PMCHL2,OrthoDBQueryError,0,No OrthoDB results for query
RPL112,OrthoDBQueryError,0,No OrthoDB results for query
SLC6A10P,OrthoDBQueryError,0,No OrthoDB results for query
SMAD14,OrthoDBQueryError,0,No OrthoDB results for query


=========AAAS=========
=========ABHD17C=========
=========ACADM=========
=========ACTR3=========
=========ADCK5=========
=========AGBL1=========
=========AGO3=========
=========AK2=========
=========AKAP1=========
=========AKR1A1=========
=========ALDOA=========
=========AMDHD2=========
=========ANAPC4=========
=========ANXA2=========
=========APEX1=========
=========ARHGAP21=========
=========ARL6IP1=========
=========ARPC1B=========
=========ARVCF=========
=========ASAH1=========
=========ASCC2=========
=========ASNS=========
=========ATF1=========
ATF1	SequenceDataError	6	No AGS NCBI sequences available.

=========ATP5G1=========
=========ATP5G3=========
=========ATP6V0B=========
=========ATPIF1=========
=========ATRX=========
=========ATXN7L1=========
=========BCAS1=========
=========BRD4=========
=========BUB3=========
=========C14ORF132=========
=========C19ORF53=========
=========C6ORF165=========
=========C7ORF57=========
=========CACB1=========
CACB1	OrthoDBQueryError	0	No Ort

=========RPS10-NUDT3=========
RPS10-NUDT3	SequenceDataError	6	No AGS NCBI sequences available.

=========RPS12=========
RPS12	SequenceDataError	6	No AGS NCBI sequences available.

=========RPS16=========
=========RPS19BP1=========
=========RPS21=========
=========RPS29=========
=========S100A4=========
=========SARS=========
=========SCAMP2=========
=========SCAPER=========
=========SCML2=========
=========SCN7A=========
=========SENP8=========
=========SEPT6=========
=========SERF2=========
=========SGF29=========
=========SIRT1=========
=========SLC22A13=========
=========SLC22A14=========
=========SLC25A3=========
=========SLC25A5=========
=========SLC35B2=========
=========SLC44A2=========
=========SLC4A2=========
=========SLC6A10P=========
SLC6A10P	OrthoDBQueryError	0	No OrthoDB results for query
=========SLC6A8=========
=========SLIRP=========
=========SLTM=========
=========SMAD14=========
SMAD14	OrthoDBQueryError	0	No OrthoDB results for query
=========SNAI2=========
=========S

,Gene,MSA Position,Test Species Position,Test Variant,Test Variant Instances,AGS Variant,AGS 13LGS Consensus,Outgroup Variant,Outgroup Variant Instances,Aligned Sequences,Gap Fraction,JSD,JSD Z-Score,Test vs Outgroup Blosum62,Outgroup Pairwise Blosum62
0,AAAS,18,18,Q,1,Q,True,L,10,11,0.000000,0.773172,-0.001767,-2.0,4.000000
1,AAAS,30,30,D,1,D,True,S,10,11,0.000000,0.836597,0.333600,0.0,4.000000
2,AAAS,58,44,D,1,D,True,N,10,11,0.000000,0.876198,0.542996,1.0,6.000000
3,AAAS,61,47,I,1,I,True,V,10,11,0.000000,0.810426,0.195221,3.0,4.000000
4,AAAS,64,50,P,1,P,True,L,10,11,0.000000,0.773172,-0.001767,-3.0,4.000000
5,AAAS,65,51,N,1,N,True,T,10,11,0.000000,0.845023,0.378155,0.0,5.000000
6,AAAS,68,54,L,1,L,True,P,10,11,0.000000,0.871565,0.518499,-3.0,7.000000
7,AAAS,77,63,Q,1,Q,True,H,10,11,0.000000,0.916058,0.753762,0.0,8.000000
8,AAAS,78,64,S,1,S,True,G,10,11,0.000000,0.789534,0.084753,0.0,6.000000
9,AAAS,193,171,Q,1,Q,True,L,10,11,0.000000,0.773172,-0.001767,-2.0,4.000000


In [ ]:
# display(tax_table)
# print(len(tax_table))

In [ ]:
# #TEST CELL
# # print(failed_searches)
# fasta_path,fasta_df, ref_index = read_query_input("ATP5G2", "cDNA_screen_ATP5G")
# # display(fasta_df)
# tsv_path = run_name+"/input/"+gene_name+".tsv"
# fasta_path = run_name+"/input/"+gene_name+".fasta"
# tsv_df = pd.read_csv(tsv_path,delimiter='\t')
# tsv_df.set_index(keys="int_prot_id",drop=True,inplace=True)#drop=False)
# fasta_seqs = SeqIO.parse(open(fasta_path),'fasta')
# fasta_df = pd.DataFrame(columns=["id","pubgene_id","desc","aaseq","length(aa)"])
# #Generate DataFrame of all reference sequences to use for input filtering. See above 
# #methods for implementation.
# ref_ids, gc_name = find_ref_seqs(gene_name,tsv_df,errors_fpath)
# print(ref_ids)

In [86]:
gene_list2 = parse_genes(genes_file)
print(len(gene_list2))

AGS_aln_base_dir = "{0}/allspec_AGS_alignments".format(run_name)
complete_analysis = []

for gene_name in gene_list2:
    AGS_gene_dir = "{0}/{1}".format(AGS_aln_base_dir,gene_name)
    if os.path.exists(AGS_gene_dir):
        complete_analysis.append(gene_name)
print(len(complete_analysis))

387
356
